In [ ]:
import re
import pickle
import nltk
from nltk.tokenize import  TweetTokenizer
from nltk.corpus import stopwords

In [ ]:
from nltk.stem.snowball import SnowballStemmer

tokenizer =  TweetTokenizer()
stop = set(stopwords.words('english')+['<user>','<url>',',','-','.','[',']',':','&'])
stemmer = SnowballStemmer("english");
mojitable = pickle.load(open('emojtable.😃', 'rb'))

prog = re.compile("(?:\s|^)((?:[^\s][^a-zA-Z1-9\n]+)(?:[^\s](?:[^a-zA-Z1-9\n]+|$))+)")
lineHead = re.compile("^(\d+,)");

def replaceMoji(s):
    splitted = prog.split(s)
    res = ""
    for i in range(0,int(len(splitted)/2)):
        emo = splitted[2*i+1].strip()
        parsedEmo = mojitable.get(emo,emo)
        if parsedEmo == "":
            parsedEmo = emo
        splitted[2*i+1] = " "+parsedEmo+" "
    
    return "".join(splitted).strip();
def prepareOneLine(line):
    try :
        tweet = line.decode('utf-8')
    except UnicodeDecodeError :
        tweet = str(line)[2:-5]
        print("cant parse "+tweet)
    head = lineHead.search(tweet)
    header = ""
    if head != None :
        header = head.group(1)
        tweet = tweet[len(header):]
    return header+preparOneTweets(tweet)
            
def preparOneTweets(tweet):    
    return  " ".join([stemmer.stem(replaceMoji(i)) for i in tokenizer.tokenize(tweet) if i not in stop])
def preprocessFile(file):
    lines = []
    with open(file,'rb') as f :
        lines = map(prepareOneLine,f.readlines())
        f.close()
    with open("cleaned"+file,"w") as f:
        f.write("\n".join(list(lines)))
        f.close()
        print("done "+file)

preparOneTweets("1-3 vs celtic regular season = fuck play playoff")

replaceMoji("guess text want us back ( ( (")

In [ ]:
files = ["train_neg.txt","train_neg_full.txt","train_pos.txt","train_pos_full.txt","test_data.txt"]
for f in files:
    preprocessFile(f)